<a href="https://colab.research.google.com/github/b21renu/NVIDIA/blob/main/NVIDIA_NIM_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain_nvidia_ai_endpoints

In [ ]:
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings, ChatNVIDIA

In [ ]:
!pip install langchain-community langchain-text-splitters faiss-cpu

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")

docs = loader.load()

In [ ]:
# ID
# 1a2109c1-e7e0-4762-910d-0577d2356284

# Name
# __autogenerated_playgrounds_0569634548873286

# Expiration
# 11/05/2025

# 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
# Keep your key a secret. Do not share it or store it in a place where others can see or copy it.

In [ ]:
from google.colab import userdata
import os
os.environ['NVIDIA_API_KEY'] = userdata.get('NVIDIA_API_KEY')

In [ ]:
embeddings = NVIDIAEmbeddings()

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)
retriever = vector.as_retriever()

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


model = ChatNVIDIA(model="mistral_7b")

/usr/local/lib/python3.10/dist-packages/langchain_nvidia_ai_endpoints/_statics.py:745: UserWarning: Model mistral_7b is deprecated. Using mistralai/mistral-7b-instruct-v0.2 instead.
  warnings.warn(


In [ ]:
model

ChatNVIDIA(base_url='https://integrate.api.nvidia.com/v1', model='mistralai/mistral-7b-instruct-v0.2')

In [ ]:
hyde_template = """Even if you do not know the full answer, generate a one-paragraph hypothetical answer to the below question:

{question}"""
hyde_prompt = ChatPromptTemplate.from_template(hyde_template)
hyde_query_transformer = hyde_prompt | model | StrOutputParser()

In [ ]:
hyde_query_transformer

ChatPromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='Even if you do not know the full answer, generate a one-paragraph hypothetical answer to the below question:\n\n{question}'), additional_kwargs={})])
| ChatNVIDIA(base_url='https://integrate.api.nvidia.com/v1', model='mistralai/mistral-7b-instruct-v0.2')
| StrOutputParser()

In [ ]:
from langchain_core.runnables import chain

@chain
def hyde_retriever(question):
    hypothetical_document = hyde_query_transformer.invoke({"question": question})
    return retriever.invoke(hypothetical_document)

In [ ]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
# answer_chain = prompt | model | StrOutputParser()
answer_chain = prompt | model

In [ ]:
@chain
def final_chain(question):
    documents = hyde_retriever.invoke(question)
    # for s in answer_chain.stream({"question": question, "context": documents}):
    #     yield s
    for s in answer_chain.stream({"question": question, "context": documents}, include_usage=False):
        yield s


In [ ]:
# for s in final_chain.stream("how can langsmith help with testing"):
#     print(s, end="")

Exception: [400] Bad Request
[{'type': 'extra_forbidden', 'loc': ('body', 'stream_options'), 'msg': 'Extra inputs are not permitted', 'input': {'include_usage': True}, 'url': 'https://errors.pydantic.dev/2.6/v/extra_forbidden'}]

In [ ]:
# def final_chain(question):
#     documents = hyde_retriever.invoke(question)
#     response = answer_chain.invoke({"question": question, "context": documents})
#     return response

# response = final_chain("how can langsmith help with testing")
# print(response)

content=' Langsmith is a platform that assists in LLM ( Large-Scale Language Model) application development, monitoring, and testing. It supports various workflows that fit into each stage of the application development lifecycle. Specifically for testing, Langsmith allows users to:\n\n1. Upload test cases in bulk, create them on the fly, or export them from application traces.\n2. Run custom evaluations, both LLM and heuristic-based, to score test results.\n3. Use a playground environment for rapid iteration and experimentation, allowing for quick testing of different prompts and models.\n4. Gather human feedback on application responses and attach feedback scores to logged traces, enabling filtering on traces with specific feedback tags and scores for further analysis and issue resolution.' additional_kwargs={} response_metadata={'role': 'assistant', 'content': ' Langsmith is a platform that assists in LLM ( Large-Scale Language Model) application development, monitoring, and testing

In [ ]:
def final_chain(question):
    documents = hyde_retriever.invoke(question)
    response = answer_chain.invoke({"question": question, "context": documents})

    # Print the response to inspect its structure
    print("Response Object:", response)  # Inspect the response structure
    print("Response Attributes:", dir(response))  # List available attributes

    # Access the appropriate attribute (modify based on the actual response structure)
    # Example: If the response has a 'content' attribute or a similar one, use that
    response_text = response.content if hasattr(response, 'content') else str(response)

    # Split the response into points based on newlines or bullet points
    points = response_text.split('\n')  # Adjust split logic based on response format

    # Clean up the points (e.g., removing empty strings)
    points = [point.strip() for point in points if point.strip()]

    return points

response = final_chain("how can langsmith help with testing")
for idx, point in enumerate(response, start=1):
    print(f"{point}")


Response Object: content=' Langsmith is a platform that assists in LLM ( Large-Scale Language Model) application development, monitoring, and testing. It supports various workflows that fit into each stage of the application development lifecycle. Specifically for testing, Langsmith allows users to:\n\n1. Upload test cases in bulk, create them on the fly, or export them from application traces.\n2. Run custom evaluations, both LLM and heuristic-based, to score test results.\n3. Use a playground environment for rapid iteration and experimentation, allowing for quick testing of different prompts and models.\n4. Gather human feedback on application responses and attach feedback scores to logged traces, enabling filtering on traces with specific feedback tags and scores for further analysis and issue resolution.' additional_kwargs={} response_metadata={'role': 'assistant', 'content': ' Langsmith is a platform that assists in LLM ( Large-Scale Language Model) application development, monito